In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Data Overview

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting which albums to purchase

In [3]:
%%sql
SELECT *
  FROM album
 LIMIT 5;

 * sqlite:///chinook.db
Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [4]:
%%sql
SELECT *
  FROM employee
 LIMIT 5;

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [5]:
%%sql
SELECT *
  FROM genre
 LIMIT 5;

 * sqlite:///chinook.db
Done.


genre_id,name
1,Rock
2,Jazz
3,Metal
4,Alternative & Punk
5,Rock And Roll


In [6]:
%%sql
SELECT *
  FROM track
 LIMIT 5;

 * sqlite:///chinook.db
Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [7]:
%%sql
SELECT *
  FROM invoice_line
 LIMIT 5;

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


In [8]:
%%sql
WITH genres_USA AS
                (
                SELECT il.invoice_id invoice_id,
                       il.track_id track_id,
                       iv.billing_country country,
                       t.genre_id genre_id,
                       g.name genre
                  FROM invoice_line il
                  LEFT JOIN invoice iv ON il.invoice_id = iv.invoice_id
                  LEFT JOIN track t on il.track_id = t.track_id
                  LEFT JOIN genre g on t.genre_id = g.genre_id
                 WHERE iv.billing_country = 'USA'
                )
SELECT genre, COUNT(genre) AS 'Total Sold',
       ROUND((CAST(COUNT(genre) AS Float) * 100) / (SELECT COUNT(country) FROM genres_USA), 2) AS 'Percentage'
  FROM genres_USA
 GROUP BY genre
 ORDER BY Percentage DESC;

 * sqlite:///chinook.db
Done.


genre,Total Sold,Percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Pop,22,2.09
Latin,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


Top genre is Rock, etc.
The three artists that we should purchase are:
 - Red Tone	Punk, Punk
 - Meteor and the Girls, Pop
 - Slim Jim Bites, Blues
 
These three genres combined barely make up 18% of total sales, as a store, they should really be looking into aquiring more albums in the Rock genre.

## Analyzing Employee sales performance

In [9]:
%%sql
WITH spending_per_customer AS
                           (
                           SELECT customer_id, SUM(total) total_spent
                             FROM invoice
                            GROUP BY customer_id
                           )
SELECT e.first_name || ' ' || e.last_name AS name,
       e.hire_date AS date_hired,
       ROUND(SUM(spc.total_spent), 2) AS total
  FROM spending_per_customer AS spc
 INNER JOIN customer c ON spc.customer_id = c.customer_id
 INNER JOIN employee e on c.support_rep_id = e.employee_id
 GROUP BY name
 ORDER BY total DESC;

 * sqlite:///chinook.db
Done.


name,date_hired,total
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


Jane Peacock has the most in sales amounts, however she is the longest working employee out of the three.

Steve Johnson brought in the least amount of money, but is working the shortest amount of time, almost 6 months less than Jane Peacock.

## Analyzing Sales By Country

In [10]:
%%sql
DROP VIEW IF EXISTS customer_spending;
CREATE VIEW customer_spending AS
                SELECT customer_id, SUM(total) AS total,
                       COUNT(customer_id) no_orders
                  FROM invoice
                 GROUP BY customer_id;

 * sqlite:///chinook.db
Done.
Done.


[]

In [11]:
%%sql
select * from customer_spending LIMIT 5;

 * sqlite:///chinook.db
Done.


customer_id,total,no_orders
1,108.89999999999998,13
2,82.17,11
3,99.99,9
4,72.27000000000001,9
5,144.54000000000002,18


In [12]:
%%sql
WITH country_stats AS
                   (
                   SELECT COUNT(c.customer_id) total_customers,
                           CASE WHEN (
                                     SELECT count(*)
                                     FROM customer
                                     where country = c.country
                                    ) = 1 THEN "Other"
                                ELSE c.country 
                            END AS country,
                           ROUND(SUM(cs.total), 2) total_sales_value,
                           ROUND(SUM(cs.total) / COUNT(c.customer_id), 2) avg_lifetime_sales,
                           ROUND(SUM(cs.total) / SUM(cs.no_orders), 2) avg_order_value,
                           CASE WHEN(
                                     SELECT count(*)
                                     FROM customer
                                     where country = c.country
                                    ) = 1 THEN 1
                                ELSE 0
                            END AS sorting_order
                      FROM customer c
                     INNER JOIN customer_spending cs ON c.customer_id = cs.customer_id
                     GROUP BY 2
                   )
SELECT country, total_customers, 
       total_sales_value, avg_lifetime_sales, 
       avg_order_value
  FROM country_stats
 ORDER BY sorting_order ASC, total_sales_value DESC;

 * sqlite:///chinook.db
Done.


country,total_customers,total_sales_value,avg_lifetime_sales,avg_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


## Album Purchases vs. Individual Track Purchases

In [13]:
%%sql
WITH invoice_track_album AS
     (
         SELECT il.invoice_line_id invoice_line_id,
                il.invoice_id invoice_id,
                il.track_id track_id,
                t.album_id album_id
           FROM invoice_line il
         LEFT JOIN track t ON il.track_id = t.track_id
     ),
unique_album_first_track AS
     (
         SELECT ita.invoice_id invoice_id,
                MIN(ita.track_id) first_track_id,
                ita.album_id album_id
           FROM invoice_track_album ita
          GROUP BY 1,3
     ),
album_per_invoice AS
     (
         SELECT uaft.*,
           CASE 
            WHEN (
                  SELECT t.track_id FROM track t
                   WHERE t.album_id = uaft.album_id
                  EXCEPT
                  SELECT ita.track_id FROM invoice_track_album ita
                   WHERE ita.invoice_id = uaft.invoice_id
                     AND ita.album_id = uaft.album_id
                 ) IS NULL
             AND (
                  SELECT ita.track_id FROM invoice_track_album ita
                   WHERE ita.invoice_id = uaft.invoice_id
                     AND ita.album_id = uaft.album_id
                  EXCEPT
                  SELECT t.track_id FROM track t
                   WHERE t.album_id = uaft.album_id
                 ) IS NULL
             AND (
                  SELECT COUNT(*) FROM track t
                   WHERE t.album_id = uaft.album_id
                 ) > 2
            THEN "YES"
            ELSE "NO"
             END AS album_purchase
           FROM unique_album_first_track uaft
          GROUP BY 1
     )
SELECT api.album_purchase,
    COUNT(api.invoice_id) number_of_invoices,
    CAST(count(api.invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
  FROM album_per_invoice api
 GROUP BY 1;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
NO,503,0.8192182410423453
YES,111,0.18078175895765472


We can see that album purchases make up just slightly over 18%, thats about one fifth of the Chinook store revenue. Stopping the purchase of albums in favour of individual tracks from record companies would result in loss of revenue.